# Benchmark results reporting

## Setup

### Prerequirements
This notebook requires a kernel running Python 3.5+.
You can skip this section if the kernel is already configured.

In [ ]:
!pip install -r ./requirements.txt
#!pip install jupyter_contrib_nbextensions
#!jupyter contrib nbextension install --user
#!jupyter nbextension enable python-markdown/main
#!pip install jupyter_nbextensions_configurator
#!jupyter nbextensions_configurator enable --user

### Imports and selection of the results directory

In [ ]:
import report
import report.config as config
from report import create_file, display

## Results

##### Parameters

In [ ]:
%run ./reports_config.py

In [ ]:
from report.config import *

#### Loading results, formatting and adding columns
- `result` is the raw result metric computed from predictions at the end the benchmark.
    For classification problems, it is usually `auc` for binomial classification and `logloss` for multinomial classification.
- `score` ensures a standard comparison between tasks: **higher is always better**.
- `norm_score` is a normalization of `score` on a `[0, 1]` scale, with `{{zero_one_refs[0]}}` score as `0` and `{{zero_one_refs[1]}}` score as `1`.
- `imp_result` and `imp_score` for imputed results/scores. Given a task and a framework:
    - if **all folds results/scores are missing**, then no imputation occurs, and the result is `nan` for each fold.
    - if **only some folds results/scores are missing**, then the missing result is imputed by the `{{impute_missing_with}}` result for this fold.

In [ ]:
from report import prepare_results

# load result files
res = prepare_results(results_files,
                      renamings=renamed_frameworks,
                      exclusions=excluded_frameworks,
                      imputation=impute_missing_with,
                      normalization=zero_one_refs)
res.results.to_csv(create_file("tables", results_group, "all_results.csv"), 
                   index=False, 
                   float_format=config.ff)

### Tasks list

In [ ]:
from report import render_metadata

# tasks = (res.results.groupby(['task', 'type'])['id']
#                     .unique()
#                     .map(lambda id: id[0]))
# display(tasks)

render_metadata(res.metadata, 
                filename=create_file("datasets", results_group, "metadata.csv"))

### Completed tasks/folds

In [ ]:
done = (res.done.reset_index()
                .groupby(['task', 'framework'])['fold']
                .unique())
display(done, pretty=False)
# display(tabulate(done, tablefmt='plain'))

### Missing or crashed/aborted tasks/folds

In [ ]:
# not_done = pd.DataFrame([(task, framework) for task in res.tasks 
#                                            for framework in res.frameworks 
#                                            if (task, framework) not in done.index],
#                         columns=['task', 'framework'])
# missing = res.results.append(not_done)\
#                      .groupby(['task', 'framework'])['fold']\
#                      .unique()\
#                      .map(sorted_ints)\
#                      .map(lambda arr: sorted(list(set(range(0, nfolds)) - set(arr))))\
#                      .where(lambda values: values.map(lambda arr: len(arr) > 0))\
#                      .dropna()

missing = (res.missing.reset_index()
                      .groupby(['task', 'framework'])['fold']
                      .unique())
display(missing, pretty=False)

### Failing tasks/folds

In [ ]:
# failed = res.results.where(np.isnan(all_results.result))\
#                     .groupby(['task', 'framework'])['fold']\
#                     .unique()\
#                     .map(sorted_ints)

failed = (res.failed.reset_index()
                    .groupby(['task', 'framework'])['fold']
                    .unique())
display(failed, pretty=False)

### Results anomalies

In [ ]:
from report.analysis import list_outliers

display(list_outliers('result', 
                      results=res.results,
#                       results=res.results.loc[res.results.framework=='h2oautoml']
                      z_threshold=2.5,
                     ))

## Data reports

### Results summary

Averaging using arithmetic mean over fold `result` or `score`.
In following summaries, if not mentioned otherwise, the means are computed over imputed results/scores.
Given a task and a framework:
- if **all folds results/scores are missing**, then no imputation occured, and the mean result is `nan`.
- if **only some folds results/scores are missing**, then the amount of imputed results that contributed to the mean are displayed between parenthesis.

In [ ]:
from report import render_summary

summary_results = res.results

#### Number of models trained

When available, displays the average amount of models trained by the framework for each dataset.

This amount should be interpreted differently for each framework.
For example, with *RandomForest*, this amount corresponds to the number of trees.

In [ ]:
render_summary('models', 
               results=summary_results, 
               filename="models_summary.csv", 
               float_format="%.f")

#### Results mean

In [ ]:
render_summary('result', 
               results=summary_results)

In [ ]:
render_summary('imp_result', 
               results=summary_results,
               filename="result_summary.csv")

#### Score mean

In [ ]:
render_summary('imp_score', 
               results=summary_results,
               filename="score_summary.csv")

In [ ]:
render_summary('norm_score', 
               results=summary_results,
               filename="norm_score_summary.csv")

### Tasks leaderboard

In [ ]:
from report import render_leaderboard

leaderboard_results = res.results.loc[~res.results.framework.isin(['constantpredictor', 'randomforest'])]

In [ ]:
render_leaderboard('imp_score', 
                   results=leaderboard_results,
                   aggregate=True, 
                   show_imputations=True, 
                   filename="tasks_leaderboard.csv")

### Folds leaderboard

In [ ]:
render_leaderboard('score', 
                   results=res.results,
                   filename="folds_leaderboard.csv");

## Visualizations

### Heatmaps

In [ ]:
from report import draw_score_heatmap

# heatmap_results = res.results.loc[~res.results.framework.isin(['constantpredictor', 'randomforest'])]
heatmap_results = res.results.loc[~res.results.framework.isin(['constantpredictor'])]

In [ ]:
draw_score_heatmap('imp_score',
                   results=heatmap_results,
                   type_filter='binary', 
                   metadata=res.metadata,
                   y_sort_by=tasks_sort_by,
                   title=f"Scores ({binary_score_label}) on {results_group} binary classification problems",
                   filename="binary_score_heat.png",
                   center=0.5);

In [ ]:
draw_score_heatmap('imp_score', 
                   results=heatmap_results,
                   type_filter='multiclass', 
                   y_sort_by=tasks_sort_by,
                   title=f"Scores ({multiclass_score_label}) on {results_group} multi-class classification problems",
                   filename="multiclass_score_heat.png",
                   center=0);

In [ ]:
draw_score_heatmap('norm_score', 
                   results=heatmap_results,
                   type_filter='binary', 
                   y_sort_by=tasks_sort_by,
                   title=f"Normalized scores on {results_group} binary classification problems",
                   filename="binary_norm_score_heat.png",
                   center=0);

In [ ]:
draw_score_heatmap('norm_score',
                   results=heatmap_results,
                   type_filter='multiclass', 
                   y_sort_by=tasks_sort_by,
                   title=f"Normalized scores on {results_group} multi-class classification problems",
                   filename="multiclass_norm_score_heat.png",
                   center=0);

### Linear plots

In [ ]:
from report import draw_score_parallel_coord

# parallel_coord_results = res.results.loc[~res.results.framework.isin(['randomforest'])]
parallel_coord_results = res.results

In [ ]:
draw_score_parallel_coord('imp_score',
                          results=parallel_coord_results,
                          type_filter='binary', 
                          metadata=res.metadata,
                          x_sort_by=tasks_sort_by,
                          title=f"Scores ({binary_score_label}) on {results_group} binary classification problems",
                          ylabel=binary_score_label,
                          legend_loc='lower left',
                          filename="binary_score_parallel_ccord.png"
                         );


In [ ]:
draw_score_parallel_coord('imp_score',
                          results=parallel_coord_results,
                          type_filter='multiclass',
                          metadata=res.metadata,
                          x_sort_by=tasks_sort_by,
                          title=f"Scores ({multiclass_score_label}) on {results_group} multi-class classification problems",
                          ylabel=multiclass_score_label,
                          yscale=('symlog', dict(linthreshy=0.5)),
                          legend_loc='lower left',
                          filename="multiclass_score_parallel_ccord.png"
                         );


In [ ]:
draw_score_parallel_coord('norm_score', 
                          results=parallel_coord_results,
                          type_filter='binary', 
                          metadata=res.metadata,
                          x_sort_by=tasks_sort_by,
                          title=f"Normalized scores on {results_group} binary classification problems",
                          legend_loc='lower left',
                          filename="binary_norm_score_parallel_ccord.png"
                         );


In [ ]:
draw_score_parallel_coord('norm_score', 
                          results=parallel_coord_results,
                          type_filter='multiclass',
                          metadata=res.metadata,
                          x_sort_by=tasks_sort_by,
                          title=f"Normalized scores on {results_group} multi-class classification problems",
                          filename="multiclass_norm_score_parallel_ccord.png", 
                          yscale='symlog',
                         );


### Scatterplots

In [ ]:
from report import draw_score_stripplot

# scatterplot_results = (res.results.loc[~res.results.framework.isin(['randomforest'])]
#                                   .sort_values(by=['framework']))  # sorting for colors consistency
scatterplot_results = res.results.sort_values(by=['framework'])  # sorting for colors consistency

In [ ]:
draw_score_stripplot('imp_result', 
                     results=scatterplot_results,
                     type_filter='binary', 
                     metadata=res.metadata,
                     y_sort_by=tasks_sort_by,
                     title=f"Scores ({binary_score_label}) on {results_group} binary classification problems",
                     xlabel=binary_score_label,
                     filename="binary_results_stripplot.png");

In [ ]:
draw_score_stripplot('imp_result',
                     results=scatterplot_results,
                     type_filter='multiclass', 
                     metadata=res.metadata,
                     y_sort_by=tasks_sort_by,
#                      xbound=(0,10),
                     xscale=('symlog', dict(linthreshx=0.5)),
                     title=f"Scores ({multiclass_score_label}) on {results_group} multi-class classification problems",
                     xlabel=multiclass_score_label, 
                     filename="multiclass_results_stripplot.png");

In [ ]:
draw_score_stripplot('norm_score', 
                     results=scatterplot_results,
                     type_filter='binary', 
                     metadata=res.metadata,
                     y_sort_by=tasks_sort_by,
                     xbound=(-0.2, 2),
                     xscale='linear',
                     title=f"Normalized scores on {results_group} binary classification problems",
                     filename="binary_norm_score_stripplot.png");

In [ ]:
draw_score_stripplot('norm_score', 
                     results=scatterplot_results,
                     type_filter='multiclass', 
                     metadata=res.metadata,
                     y_sort_by=tasks_sort_by,
                     xbound=(-0.2, 2.5),
                     xscale='linear',
                     title=f"Normalized scores on {results_group} multi-class classification problems",
                     filename="multiclass_norm_score_stripplot.png");

## Playground

In [ ]:
res.results.loc[(res.results.task.str.contains('jungle'))&(res.results.framework=='tunedrandomforest')];

In [ ]:
done.iloc[done.index.get_level_values('framework').isin(['autosklearn', 'h2oautoml', 'tpot'])]\
    .apply(sorted_ints);

In [ ]:
failures = res.failed.groupby(['task', 'fold', 'framework'])['info']\
                     .unique()
#display(failures)

In [ ]:
h2o_res = (res.results.loc[(res.results.framework=='h2oautoml')&(res.results.fold==0)]
                     [['task', 'result', 'acc']])
# display(h2o_res, pretty=False, tab_format='grid')
display(h2o_res)